In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from tqdm import tqdm
import sys
sys.path.append('utils/')

from image_preprocessing_ver1 import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
from keras.models import Model

from resnet import get_resnet

In [ ]:
data_dir = '/home/ubuntu/data/'

In [ ]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=lambda x: preprocess_input(np.expand_dims(x, 0))
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(224, 224),
    batch_size=64, shuffle=False
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(224, 224),
    batch_size=64, shuffle=False
)

In [ ]:
model = get_resnet()
model.load_weights('resnet_weights.hdf5')
model.layers.pop()
model = Model(model.input, model.layers[-1].output)

In [ ]:
batches = 0
train_logits = {}

for x_batch, _, name_batch in tqdm(train_generator):
    
    batch_logits = model.predict_on_batch(x_batch)
    
    for i, n in enumerate(name_batch):
        train_logits[n] = batch_logits[i]
    
    batches += 1
    if batches >= 400:
        break

In [ ]:
batches = 0
val_logits = {}

for x_batch, _, name_batch in tqdm(val_generator):
    
    batch_logits = model.predict_on_batch(x_batch)
    
    for i, n in enumerate(name_batch):
        val_logits[n] = batch_logits[i]
    
    batches += 1
    if batches >= 80:
        break

In [ ]:
np.save(data_dir + 'train_logits.npy', train_logits)
np.save(data_dir + 'val_logits.npy', val_logits)